In [47]:
import json
import pandas as pd
import numpy as np
import re
from datetime import datetime

In [48]:
pre_ca = pd.read_csv("/Users/alyssanguyen/Desktop/IRLE_scraping/csv_files/processed_prices_ubereats_ca_ff_03222024.csv")
post_ca = pd.read_csv("/Users/alyssanguyen/Desktop/IRLE_scraping/csv_files/final_processed_prices_ubereats_ca_ff_05142024.csv")

pre_nonca = pd.read_csv("/Users/alyssanguyen/Desktop/IRLE_scraping/csv_files/processed_prices_ubereats_nonca_ff_03292024.csv")
post_nonca = pd.read_csv("/Users/alyssanguyen/Desktop/IRLE_scraping/csv_files/final_processed_prices_ubereats_nonca_ff_05162024.csv")
post_nonca['post_policy'] = 1

In [49]:
combined = pd.concat([pre_ca, post_ca, pre_nonca, post_nonca])

In [50]:
combined = combined[['restaurant_name', 'hamburger', 'cheeseburger', 'specialty_item', 'state', 'post_policy']]

In [51]:
combined = combined.copy()
combined['treatment'] = combined['state'].apply(lambda x: 1 if x == 'ca' else 0)
combined['interaction'] = combined['post_policy'] * combined['treatment']
combined = combined.dropna()

In [52]:
combined

restaurant_name  hamburger  cheeseburger  specialty_item state  \
0            McDonald       3.49          4.39            7.89    ca   
1         Burger King       5.39          2.79           10.29    ca   
2            McDonald       3.89          4.39            8.29    ca   
3            McDonald       2.95          3.04            6.46    ca   
4     Jack in the Box       3.11          3.74            6.86    ca   
...               ...        ...           ...             ...   ...   
2247        Five Guys       7.91          8.87           11.63    ky   
2248        Five Guys       9.71         11.15           13.79    pa   
2249        Five Guys       9.23         10.55           13.55    ok   
2250        Five Guys       9.35         10.67           13.67    tx   
2251        Five Guys       9.35         10.67           13.67    al   

      post_policy  treatment  interaction  
0               0          1            0  
1               0          1            0  
2               0          1            0  
3               0          1            0  
4               0          1            0  
...           ...        ...          ...  
2247            1          0            0  
2248            1          0            0  
2249            1          0            0  
2250            1          0            0  
2251            1          0            0  

[4912 rows x 8 columns]

In [56]:
import statsmodels.api as sm
import statsmodels.formula.api as smf

# Initialize an empty dictionary to store results
results = {}

# Get unique restaurant chains
chains = combined['restaurant_name'].unique()

# List of items to analyze
items = ['hamburger', 'cheeseburger', 'specialty_item']

for chain in chains:
    # Filter data for the current chain
    chain_data = combined[combined['restaurant_name'] == chain]
    
    # Initialize dictionary to store results for each item
    results[chain] = {}
    
    for item in items:
        # Define the model for the current item
        formula = f'{item} ~ post_policy * treatment'
        model = smf.ols(formula, data=chain_data).fit()
        
        # Store the results
        results[chain][item] = model

# Print the results for each chain
for chain, chain_results in results.items():
    print(f"Results for {chain}:")
    for item, model in chain_results.items():
        print(f"  {item}:")
        print(model.summary())
        print("\n")

Results for McDonald:
  hamburger:
                            OLS Regression Results                            
Dep. Variable:              hamburger   R-squared:                       0.551
Model:                            OLS   Adj. R-squared:                  0.550
Method:                 Least Squares   F-statistic:                     519.2
Date:                Mon, 22 Jul 2024   Prob (F-statistic):          3.42e-220
Time:                        16:20:30   Log-Likelihood:                -833.26
No. Observations:                1275   AIC:                             1675.
Df Residuals:                    1271   BIC:                             1695.
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                            coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------

/Users/alyssanguyen/opt/anaconda3/lib/python3.9/site-packages/scipy/stats/_stats_py.py:1806: UserWarning: kurtosistest only valid for n>=20 ... continuing anyway, n=14
  warnings.warn("kurtosistest only valid for n>=20 ... continuing "
/Users/alyssanguyen/opt/anaconda3/lib/python3.9/site-packages/statsmodels/regression/linear_model.py:1918: RuntimeWarning: divide by zero encountered in scalar divide
  return np.sqrt(eigvals[0]/eigvals[-1])
/Users/alyssanguyen/opt/anaconda3/lib/python3.9/site-packages/scipy/stats/_stats_py.py:1806: UserWarning: kurtosistest only valid for n>=20 ... continuing anyway, n=14
  warnings.warn("kurtosistest only valid for n>=20 ... continuing "
/Users/alyssanguyen/opt/anaconda3/lib/python3.9/site-packages/statsmodels/regression/linear_model.py:1918: RuntimeWarning: divide by zero encountered in scalar divide
  return np.sqrt(eigvals[0]/eigvals[-1])
/Users/alyssanguyen/opt/anaconda3/lib/python3.9/site-packages/scipy/stats/_stats_py.py:1806: UserWarning: kurtos

In [58]:
summary_df = pd.DataFrame()

# Loop through each chain and each item
for chain, models in results.items():
    for item, model in models.items():
        # Append summary statistics for each item and chain
        summary_df = summary_df.append({
            'Chain': chain,
            'Item': item,
            'Coef_DiD': model.params['post_policy:treatment'],
            'StdErr_DiD': model.bse['post_policy:treatment'],
            'CI_Lower_DiD': model.conf_int().loc['post_policy:treatment'][0],
            'CI_Upper_DiD': model.conf_int().loc['post_policy:treatment'][1]
        }, ignore_index=True)


# Print the summary DataFrame
summary_df

/var/folders/p9/_x396g015hb1f7h6k_mytrnc0000gn/T/ipykernel_16957/3612501037.py:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  summary_df = summary_df.append({
/var/folders/p9/_x396g015hb1f7h6k_mytrnc0000gn/T/ipykernel_16957/3612501037.py:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  summary_df = summary_df.append({
/var/folders/p9/_x396g015hb1f7h6k_mytrnc0000gn/T/ipykernel_16957/3612501037.py:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  summary_df = summary_df.append({
/var/folders/p9/_x396g015hb1f7h6k_mytrnc0000gn/T/ipykernel_16957/3612501037.py:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  summary_df = summary_df.append({


Chain            Item  Coef_DiD  StdErr_DiD  CI_Lower_DiD  \
0          McDonald       hamburger  0.102637    0.057343     -0.009861   
1          McDonald    cheeseburger  0.145402    0.063369      0.021083   
2          McDonald  specialty_item  0.161407    0.092842     -0.020734   
3       Burger King       hamburger  1.721051    0.242784      1.244575   
4       Burger King    cheeseburger  0.526019    0.070805      0.387059   
5       Burger King  specialty_item -0.331697    0.130171     -0.587164   
6   Jack in the Box       hamburger -0.134321    0.102089     -0.334725   
7   Jack in the Box    cheeseburger  0.029464    0.061371     -0.091009   
8   Jack in the Box  specialty_item -0.124157    0.211165     -0.538680   
9         The Habit       hamburger  0.378643    0.062918      0.254842   
10        The Habit    cheeseburger  0.381513    0.062775      0.257992   
11        The Habit  specialty_item  0.476146    0.094656      0.289894   
12            Wendy       hamburger  0.048927    0.059784     -0.068408   
13            Wendy    cheeseburger  0.069620    0.058972     -0.046121   
14            Wendy  specialty_item  0.322602    0.096272      0.133655   
15        Five Guys       hamburger  0.060198    0.113688     -0.163152   
16        Five Guys    cheeseburger  0.170522    0.137984     -0.100558   
17        Five Guys  specialty_item  0.147351    0.151698     -0.150672   
18      Shake Shack       hamburger -0.016061    0.066958     -0.148115   
19      Shake Shack    cheeseburger -0.014545    0.071362     -0.155286   
20      Shake Shack  specialty_item -0.012121    0.060041     -0.130535   
21         Carls Jr       hamburger  0.000000    0.000000      0.000000   
22         Carls Jr    cheeseburger  0.000000    0.000000      0.000000   
23         Carls Jr  specialty_item  0.000000    0.000000      0.000000   

    CI_Upper_DiD  
0       0.215135  
1       0.269722  
2       0.343548  
3       2.197528  
4       0.664978  
5      -0.076229  
6       0.066083  
7       0.149938  
8       0.290366  
9       0.502444  
10      0.505033  
11      0.662398  
12      0.166263  
13      0.185362  
14      0.511549  
15      0.283548  
16      0.441602  
17      0.445374  
18      0.115994  
19      0.126195  
20      0.106292  
21      0.000000  
22      0.000000  
23      0.000000

In [55]:
results

{'McDonald': {'hamburger': <statsmodels.regression.linear_model.RegressionResultsWrapper at 0x7f88d28d9910>,
  'cheeseburger': <statsmodels.regression.linear_model.RegressionResultsWrapper at 0x7f88d28d9280>,
  'specialty_item': <statsmodels.regression.linear_model.RegressionResultsWrapper at 0x7f88d28df640>},
 'Burger King': {'hamburger': <statsmodels.regression.linear_model.RegressionResultsWrapper at 0x7f88d28e5370>,
  'cheeseburger': <statsmodels.regression.linear_model.RegressionResultsWrapper at 0x7f88d28e9910>,
  'specialty_item': <statsmodels.regression.linear_model.RegressionResultsWrapper at 0x7f88d28f2910>},
 'Jack in the Box': {'hamburger': <statsmodels.regression.linear_model.RegressionResultsWrapper at 0x7f88d28f73a0>,
  'cheeseburger': <statsmodels.regression.linear_model.RegressionResultsWrapper at 0x7f88881950a0>,
  'specialty_item': <statsmodels.regression.linear_model.RegressionResultsWrapper at 0x7f888819eee0>},
 'The Habit': {'hamburger': <statsmodels.regression.li

In [ ]:
def mean_non_zero(series):
    non_zero_values = series[series != 0]
    if not non_zero_values.empty:
        mean_value = non_zero_values.mean()
        return mean_value if not np.isnan(mean_value) else 0
    else:
        return 0

#restaurants = pre_filtered['restaurant_name'].unique().tolist()

#Has Hamburger
#restaurants = ['McDonald', 'Burger King', 'Jack in the Box', 'The Habit', 'Wendy', 'Five Guys', 'Shake Shack']
restaurants = ['Outback Steakhouse', 'Applebee']



# location_ca = {
# #     'McDonald': 1221, 
# #     'Burger King': 534, 
# #     'Jack in the Box': 942, 
# #     'The Habit': 258, dd
# #     'Wendy': 294, 
# #    # 'Carls Jr': 655, 
# #     'Five Guys': 124, 
# #     'Shake Shack': 44,
# #    # 'Sonic': 82,
#     'Outback Steakhouse': 44, 
#     #'Red Robin': 57,
#     #'Denny': 389,
#     'Applebee': 106, 
#     #'Buffalo Wild Wings': 96, 
#     #'TGI Fridays': 16 
# }

location_nonca = {
#     'McDonald': 12308, 
#     'Burger King': 6193, 
#     'Jack in the Box': 1251, 
#     'The Habit': 109, 
#     'Wendy': 5728, 
# #     'Carls Jr': 2035, 
#     'Five Guys': 1346, 
#     'Shake Shack': 284,
# #     'Sonic': 3443,
    'Outback Steakhouse': 628, 
 #   'Red Robin': 438,
  #  'Denny': 1141,
    'Applebee': 1430, 
  #  'Buffalo Wild Wings': 1195, 
   # 'TGI Fridays': 212,
}


def weighted_avg(restaurants, df, locations):
    numerator = {
        'hamburger': 0,
        'cheeseburger': 0,
        'specialty_item': 0
    }
    denominator = sum(locations.values())
    
    for restaurant in restaurants:
        print(f"restaurant: {restaurant}")
        curr_df = df[df['restaurant_name'] == restaurant]
        hamburger_avg = mean_non_zero(curr_df['hamburger']) * locations[restaurant]
        cheeseburger_avg = mean_non_zero(curr_df['cheeseburger']) * locations[restaurant]
        specialty_avg = mean_non_zero(curr_df['specialty_item']) * locations[restaurant]
        
        print(f"added hamburger: {hamburger_avg}")
        print(f"added cheeseburger: {cheeseburger_avg}")
        print(f"added specialty: {specialty_avg}")
        
        numerator['hamburger'] += hamburger_avg
        numerator['cheeseburger'] += cheeseburger_avg
        numerator['specialty_item'] += specialty_avg
        print(f"hamburger: {numerator['hamburger']}")
        print(f"cheeseburger: {numerator['cheeseburger']}")
        print(f"specialty: {numerator['specialty_item']}")
        
    weighted_avg = {key: numerator[key] / denominator for key in numerator.keys()}
        
    return weighted_avg

weighted_avg(restaurants, pre_filtered, location_nonca)